In [13]:
import sys
import os

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../"))

import desc
from desc.objectives import *
from desc.basis import ZernikePolynomial, FourierZernikeBasis, DoubleFourierSeries
from desc.transform import Transform
from desc.equilibrium import EquilibriaFamily, Equilibrium
from desc.plotting import (
    plot_1d,
    plot_2d,
    plot_3d,
    plot_comparison, 
    plot_section, 
    plot_fsa, 
    plot_surfaces,
)
from desc.optimize import Optimizer
from desc.perturbations import *
import numpy as np
np.set_printoptions(linewidth=np.inf)

from desc.geometry import (
    FourierRZToroidalSurface, 
    ZernikeRZToroidalSection, 
    SplineXYZCurve, 
    FourierXYZCurve,
)
%matplotlib inline
from desc.utils import copy_coeffs
import matplotlib.pyplot as plt
from desc.examples import get
from desc.objectives.getters import (
    get_fixed_boundary_constraints, 
    maybe_add_self_consistency,
)
from desc.grid import LinearGrid, QuadratureGrid, Grid
import plotly.graph_objects as go
from desc.profiles import PowerSeriesProfile
from desc.__main__ import main
from desc.vmec_utils import vmec_boundary_subspace
from desc.compute import data_index
from desc.coils import SplineXYZCoil
from desc.equilibrium import Equilibrium
from desc.geometry import FourierRZToroidalSurface
from desc.objectives import (
    ObjectiveFunction,
    ForceBalance,
    get_fixed_boundary_constraints,
)
from desc.optimize import Optimizer
from desc.plotting import plot_1d, plot_section, plot_surfaces
from desc.profiles import PowerSeriesProfile
from desc.examples import get
from desc.grid import LinearGrid
import plotly.graph_objects as go

from scipy.linalg import qr_insert, qr

import jax.numpy as jnp
from jax.lax import fori_loop
from jax.lax import rsqrt
import jax

import functools


In [2]:
eq = desc.examples.get("ESTELL")
eq.change_resolution(6, 6, 6, 12, 12, 12)
eq.solve(maxiter=20, ftol=0, xtol=0, gtol=0, verbose=3) 

/home/yigit/Codes/DESC/desc/utils.py:547: UserWarning: Reducing radial (L) resolution can make plasma boundary inconsistent. Recommend calling `eq.surface = eq.get_surface_at(rho=1.0)`
  warnings.warn(msg, err)


Building objective: force
Precomputing transforms
Timer: Precomputing transforms = 1.14 sec
Timer: Objective build = 3.96 sec
Building objective: lcfs R
Building objective: lcfs Z
Building objective: fixed Psi
Building objective: fixed pressure
Building objective: fixed current
Building objective: fixed sheet current
Building objective: self_consistency R
Building objective: self_consistency Z
Building objective: lambda gauge
Building objective: axis R self consistency
Building objective: axis Z self consistency
Timer: Objective build = 1.62 sec
Timer: Linear constraint projection build = 2.96 sec
Number of parameters: 375
Number of objectives: 2450
Starting optimization
Using method: lsq-exact
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1          4.299e-02                                    1.787e+01   
tall=True, Q.shape=(2450, 2450), R.shape=(2450, 375), J_a.shape=(2450, 375)
TrueFun
       1              2       

(Equilibrium at 0x74ac783de810 (L=6, M=6, N=6, NFP=2, sym=True, spectral_indexing=ansi),
     message: Maximum number of iterations has been exceeded.
     success: False
         fun: [-3.527e-06 -4.407e-06 ...  2.988e-05 -5.158e-05]
           x: [-1.903e-03  4.002e-02 ...  8.050e-03 -1.356e-01]
         nit: 20
        cost: 1.1440899678202393e-06
           v: [ 1.000e+00  1.000e+00 ...  1.000e+00  1.000e+00]
  optimality: 1.856686141946355e-05
        nfev: 21
        njev: 21
        allx: [Array([-1.823e-07,  2.599e-06, ...,  0.000e+00,  0.000e+00],      dtype=float64), Array([-1.237e-06,  2.645e-05, ...,  0.000e+00,  0.000e+00],      dtype=float64), Array([ 6.366e-06,  4.289e-06, ...,  0.000e+00,  0.000e+00],      dtype=float64), Array([ 1.451e-06, -5.923e-07, ...,  0.000e+00,  0.000e+00],      dtype=float64), Array([-2.984e-06, -5.012e-06, ...,  0.000e+00,  0.000e+00],      dtype=float64), Array([-1.199e-06, -3.893e-06, ...,  0.000e+00,  0.000e+00],      dtype=float64), Array(

# MASTER

In [2]:
eq = desc.examples.get("ESTELL")
eq.change_resolution(6, 6, 6, 12, 12, 12)
eq.solve(maxiter=20, ftol=0, xtol=0, gtol=0, verbose=3, copy=True) 

/home/yigit/Codes/DESC/desc/utils.py:547: UserWarning: Reducing radial (L) resolution can make plasma boundary inconsistent. Recommend calling `eq.surface = eq.get_surface_at(rho=1.0)`
  warnings.warn(msg, err)


Building objective: force
Precomputing transforms
Timer: Precomputing transforms = 2.90 sec
Timer: Objective build = 11.0 sec
Building objective: lcfs R
Building objective: lcfs Z
Building objective: fixed Psi
Building objective: fixed pressure
Building objective: fixed current
Building objective: fixed sheet current
Building objective: self_consistency R
Building objective: self_consistency Z
Building objective: lambda gauge
Building objective: axis R self consistency
Building objective: axis Z self consistency
Timer: Objective build = 3.89 sec
Timer: Linear constraint projection build = 7.05 sec
Number of parameters: 375
Number of objectives: 2450
Starting optimization
Using method: lsq-exact
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1          4.299e-02                                    1.787e+01   
       1              2          5.180e-04      4.247e-02      8.453e-02      2.326e+00   
       2              3

(Equilibrium at 0x7a34e68c0690 (L=6, M=6, N=6, NFP=2, sym=True, spectral_indexing=ansi),
     message: Maximum number of iterations has been exceeded.
     success: False
         fun: [-3.527e-06 -4.407e-06 ...  2.988e-05 -5.158e-05]
           x: [-1.903e-03  4.002e-02 ...  8.050e-03 -1.356e-01]
         nit: 20
        cost: 1.1440899678202514e-06
           v: [ 1.000e+00  1.000e+00 ...  1.000e+00  1.000e+00]
  optimality: 1.856688569593059e-05
        nfev: 21
        njev: 21
        allx: [Array([-1.823e-07,  2.599e-06, ...,  0.000e+00,  0.000e+00],      dtype=float64), Array([-1.237e-06,  2.645e-05, ...,  0.000e+00,  0.000e+00],      dtype=float64), Array([ 6.366e-06,  4.289e-06, ...,  0.000e+00,  0.000e+00],      dtype=float64), Array([ 1.451e-06, -5.923e-07, ...,  0.000e+00,  0.000e+00],      dtype=float64), Array([-2.984e-06, -5.012e-06, ...,  0.000e+00,  0.000e+00],      dtype=float64), Array([-1.199e-06, -3.893e-06, ...,  0.000e+00,  0.000e+00],      dtype=float64), Array(